In [1]:
import numpy as np
from rule_mining import rule_mining
from kb_extension import extend_kb
from ampligraph.latent_features import restore_model

# Rule mining comparison

In [10]:
# original knowlege base
original_kb = np.loadtxt("family_subset_test.txt", dtype = 'object')

# mine rules from original knowledge base
original_rules = rule_mining(original_kb)

In [2]:
original_kb = np.loadtxt("family_subset_test.txt", dtype = 'object')

# load pretrained knowledge graph embeddings
# TODO: add files for other models
complEx_model = restore_model('./Wikidata_family_subset_100_epocs.pkl')
distMult_model = restore_model('./Wikidata_family_subset_100_epocs.pkl')
transE_model = restore_model('./Wikidata_family_subset_100_epocs.pkl')
randomBaseline_model = restore_model('./Wikidata_family_subset_100_epocs.pkl')

In [3]:
# parameters
models = [complEx_model, distMult_model]#, transE_model, randomBaseline_model]
entity_selection_methods = ["random", "most_frequent"]#, "least_frequent"]
candidate_admittance_criteria = [1,2] # right now this is just rankt cutoff value

In [4]:
mined_rules = []
kb_extensions = []
for model in models:
    for method in entity_selection_methods:
        for criteria in candidate_admittance_criteria:
            extended_kb, admitted_candidates = extend_kb(original_kb, model, method, criteria)
            rules = rule_mining(extended_kb)
            kb_extensions.append(admitted_candidates)
            mined_rules.append(rules)

c:\users\johan\appdata\local\programs\python\python37\lib\site-packages\ampligraph\latent_features\models\EmbeddingModel.py:1329: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if corruption_entities == 'all':
100%|███████████████████████████████████████| 600/600 [00:01<00:00, 559.36it/s]


In [11]:
mined_rules[0]

,Rule,Head Coverage,PCA Confidence,Positive Examples,PCA Body size,Functional variable
0,?b sibling ?a => ?a sibling ?b,0.102941176,0.636363636,14,22,-2
1,?a spouse ?b => ?a sibling ?b,0.022058824,0.5,3,6,-2
2,?b spouse ?a => ?a sibling ?b,0.022058824,0.3,3,10,-2
3,?b mother ?a => ?a child ?b,0.014705882,0.4,2,5,-2
4,?b father ?a => ?a child ?b,0.088235294,0.8,12,15,-2
...,...,...,...,...,...,...
66,?h child ?b ?a child ?h => ?a child ?b,0.022058824,0.125,3,24,-2
67,?g father ?a ?g sibling ?b => ?a child...,0.066176471,0.75,9,12,-2
68,?g father ?a ?g spouse ?b => ?a child ?b,0.022058824,0.75,3,4,-2
69,?b father ?a ?b mother ?a => ?a child ?b,0.014705882,1,2,2,-2
